In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.0
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkContext available as sc, HiveContext available as sqlCtx.


In [2]:
# исходные данные 
user_id = 387

In [3]:
# файл с оценками
# user id | item id | rating | timestamp
user_marks_src = sc.textFile("/recsys/ml-100k/u.data")
user_marks_src.first()

u'196\t242\t3\t881250949'

In [4]:
# файл с фильмами
film_data_src = sc.textFile("/recsys/ml-100k/u.item")
film_data_src.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [5]:
# id, name
film_data = film_data_src.map(lambda x: (int(x.split('|')[0]), x.split('|')[1]))
film_data.take(5)

[(1, u'Toy Story (1995)'),
 (2, u'GoldenEye (1995)'),
 (3, u'Four Rooms (1995)'),
 (4, u'Get Shorty (1995)'),
 (5, u'Copycat (1995)')]

In [6]:
# user id | item id | rating | timestamp
user_marks = user_marks_src.map(lambda x: [int(i) for i in x.split('\t')])
user_marks.first()

[196, 242, 3, 881250949]

##Часть1. Найдем основные характеристики данных movielens

####Количество всех пользователей и количество всех фильмов в данных

In [7]:
# общее кол-во пользователей
user_count = user_marks.map(lambda x: x[0]).distinct().count()
print user_count

943


In [8]:
# кол-во фильмов
film_count = film_data.count()
print film_count

1682


####Сколько пользователь в среднем ставит рейтингов? (количество рейтингов/количество пользователей) – average_user_ratings

In [9]:
average_user_ratings = round(float(user_marks.count()) / user_count, 4)
print average_user_ratings

106.0445


####Сколько фильм в среднем имеет рейтингов (количество рейтингов/количество фильмов)? – average_film_ratings

In [10]:
average_film_ratings = round(float(user_marks.count()) / film_count, 4)
print average_film_ratings

59.453


####Процент заполненных ячеек в данных (количество рейтингов/(количество пользователей*количество фильмов)) – completeness

In [11]:
completeness = round(float(user_marks.count()) / (user_count*film_count), 4)
print completeness

0.063


##Часть 2. User-user CF

####Средний рейтинг для каждого пользователя

In [12]:
from operator import add
user_ratings_counts = user_marks.map(lambda x: (x[0], 1)).reduceByKey(add)
user_ratings_counts.take(3)

[(2, 62), (4, 24), (6, 211)]

In [13]:
user_ratings_sum = user_marks.map(lambda x: (x[0], x[2])).reduceByKey(add)
user_ratings_sum.take(3)

[(2, 230), (4, 104), (6, 767)]

In [14]:
user_avg = user_ratings_sum.join(user_ratings_counts).map(lambda x: (x[0], round(float(x[1][0]) / x[1][1], 4)))
user_avg.take(3)

[(2, 3.7097), (4, 4.3333), (6, 3.6351)]

####Корреляция Пирсона

In [15]:
# средняя оценка моего юзера
my_user_avg = user_avg.filter(lambda x: x[0] == user_id).collect()[0][1]
print my_user_avg

3.3618


In [16]:
# film_id, rating
my_user_marks = user_marks.filter(lambda x: x[0] == user_id).map(lambda x: (x[1], x[2]))
my_user_marks.count()

304

In [17]:
# общие фильмы и оценки
# film_id, (rating, my_rating)
def get_common_films(_uid):
    return user_marks.filter(lambda x: x[0] == _uid).map(lambda x: (x[1], x[2])).join(my_user_marks)

In [18]:
# средние оценки других пользователей
other_user_avg = user_avg.filter(lambda x: x[0] is not user_id)
other_user_avg.take(3)

[(2, 3.7097), (4, 4.3333), (6, 3.6351)]

In [19]:
other_user_avg_array = other_user_avg.collect()
up_list = []
print len(other_user_avg_array)

943


In [44]:
from math import sqrt
from pyspark.mllib.stat import Statistics

def per(film_item):
    global sum1
    global sum2
    global sum3
    user_film_rating = film_item[1][0]
    my_film_rating = film_item[1][1]
    sum1 += (my_film_rating - my_user_avg_br.value) * (user_film_rating - user_avg_rating_br.value)
    sum2 += (my_film_rating - my_user_avg_br.value) ** 2
    sum3 += (user_film_rating - user_avg_rating_br.value) ** 2


result = []
    
# идем по всем средним оценкам пользователей
__c = 0
for _item in other_user_avg_array:
    
    __c += 1
    
    # пользователь
    uid = _item[0]
    
    # Общие фильмы с моим пользователем
    common_films = get_common_films(uid)
    
    # средние оценки по общим фильмам
    common_films_count = common_films.count()
    my_avg = common_films.map(lambda x: (1, x[1][1])).reduceByKey(add).collect()[0][1]
    my_avg = round(float(my_avg) / common_films_count, 4)
    
    u_avg = common_films.map(lambda x: (1, x[1][0])).reduceByKey(add).collect()[0][1]
    u_avg = round(float(u_avg) / common_films_count, 4)
    
#     # через mllib 2
#     data1 = common_films.map(lambda x: x[1][0])
#     data2 = common_films.map(lambda x: x[1][1])
#     ml_res2 = round(Statistics.corr(data1, data2, method="pearson"), 4)
    
    # руками 1
    sum1 = sc.accumulator(0)
    sum2 = sc.accumulator(0)
    sum3 = sc.accumulator(0)
    
    my_user_avg_br = sc.broadcast(my_avg)
    user_avg_rating_br = sc.broadcast(u_avg)
    
    common_films.foreach(per)
    if sum2.value == 0 or sum3.value == 0:
        pears = 0
    else:        
        pears = round(float(sum1.value) / (sqrt(sum2.value) * sqrt(sum3.value)), 4)
    
#     # руками совсем 
#     common_film_list = common_films.collect()
#     s1 = 0
#     s2 = 0
#     s3 = 0
#     for film in common_film_list:
#         user_film_rating = film[1][0]
#         my_film_rating = film[1][1]
#         s1 += (my_film_rating - my_avg) * (user_film_rating - u_avg)
#         s2 += (my_film_rating - my_avg) ** 2
#         s3 += (user_film_rating - u_avg) ** 2
#     pears3 = round(float(s1) / (sqrt(s2) * sqrt(s3)), 4)
    
    _k = min(round(float(common_films_count)/50, 4), 1)
    new_pears = round(_k * pears, 4)
    
    res = (uid, pears, new_pears)
    result.append(res)
    
    if __c % 100 == 0:
        print __c * 100 / len(other_user_avg_array)
    

10
21
31
42
53
63
74
84
95


In [53]:
from operator import itemgetter
sorted_result = sorted(result, key=itemgetter(2), reverse=True)
sorted_result_30 =  sorted_result[0:31]
pearson_neighbours = map(lambda x: x[0], sorted_result_30)[1:]
print pearson_neighbours

[21, 267, 776, 541, 367, 118, 177, 397, 486, 442, 457, 77, 41, 854, 276, 339, 508, 323, 94, 23, 875, 158, 5, 622, 781, 933, 49, 548, 297, 566]


In [21]:
# сохраняем
import json

result = {
    "average_film_ratings": average_film_ratings,
    "average_rating": 0,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": pearson_neighbours,
    "pearson_top10": [],
    "predicators_positive_top10": [],
    "predicators_top10": []
}

with open('lab11.json', 'w') as f:
    f.write(json.dumps(result))
    f.close()